In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
programs = ["Informatique"]
years = ["2007-2008", "2008-2009", "2009-2010", "2010-2011", "2011-2012", "2012-2013",
         "2013-2014", "2014-2015", "2015-2016", "2016-2017"]
bachelor_semesters = ["Bachelor semestre 1", "Bachelor semestre 2", "Bachelor semestre 3", "Bachelor semestre 4",
                     "Bachelor semestre 5", "Bachelor semestre 6"]
master_semesters = ["Master semestre 1", "Master semestre 2", "Master semestre 3", "Projet Master automne", 
                   "Projet Master printemps"]


In [4]:
base_url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS"
r = requests.get(base_url+".filter", params = {"ww_i_reportModel": 133685247})
form = BeautifulSoup(r.text, 'html.parser')

In [5]:
program_values = {}
for program in programs:
    program_values[program] = form.find_all("option", string=program)[0]['value']

In [6]:
year_values = {}
for year in years:
    year_values[year] = form.find_all("option", string=year)[0]['value']

In [7]:
bachelor_semester_values = {}
for semester in bachelor_semesters:
    bachelor_semester_values[semester] = form.find_all("option", string=semester)[0]['value']

In [8]:
master_semester_values = {}
for semester in master_semesters:
    master_semester_values[semester] = form.find_all("option", string=semester)[0]['value']

In [9]:
def get_dataset(prog_val, y_val, sem_val):
    params = {"ww_i_reportModel": 133685247, 
              "ww_b_list": 1, 
              "ww_i_reportModelXsl": 133685270, 
              "ww_x_UNITE_ACAD": prog_val,
              "ww_x_PERIODE_ACAD": y_val,
              "ww_x_PERIODE_PEDAGO": sem_val}
    r = requests.get(base_url+".filter", params = params)
    set_page = BeautifulSoup(r.text, 'html.parser')
    gps_string = set_page.find_all("a", attrs={"class": "ww_x_GPS"})[1]["onclick"]
    gps_value = ''.join(list(filter(str.isdigit, gps_string)))
    params = {"ww_i_reportModel": 133685247, 
              "ww_i_reportModelXsl": 133685270,
              "ww_x_GPS": gps_value}
    r = requests.get(base_url+".html", params = params)
    res=pd.read_html(r.text, skiprows = [0], header = 0, index_col = "Nom Prénom")
    if len(res):
        return res[0].drop("Unnamed: 11", axis=1)
    else:
        return pd.DataFrame(columns = ['Civilité', 'Orientation Bachelor', 'Orientation Master', 
                                       'Spécialisation', 'Filière opt.', 'Mineur', 'Statut', 'Type Echange', 
                                       'Ecole Echange', 'No Sciper'])



In [10]:
bachelor_datasets = {}
for year in years:
    bachelor_datasets[year] = {}
for year in years:
    for semester in bachelor_semesters:
        print(year, semester)
        bachelor_datasets[year][semester] = get_dataset(program_values[programs[0]], 
                                                        year_values[year], bachelor_semester_values[semester])


2007-2008 Bachelor semestre 1
2007-2008 Bachelor semestre 2
2007-2008 Bachelor semestre 3
2007-2008 Bachelor semestre 4
2007-2008 Bachelor semestre 5
2007-2008 Bachelor semestre 6
2008-2009 Bachelor semestre 1
2008-2009 Bachelor semestre 2
2008-2009 Bachelor semestre 3
2008-2009 Bachelor semestre 4
2008-2009 Bachelor semestre 5
2008-2009 Bachelor semestre 6
2009-2010 Bachelor semestre 1
2009-2010 Bachelor semestre 2
2009-2010 Bachelor semestre 3
2009-2010 Bachelor semestre 4
2009-2010 Bachelor semestre 5
2009-2010 Bachelor semestre 6
2010-2011 Bachelor semestre 1
2010-2011 Bachelor semestre 2
2010-2011 Bachelor semestre 3
2010-2011 Bachelor semestre 4
2010-2011 Bachelor semestre 5
2010-2011 Bachelor semestre 6
2011-2012 Bachelor semestre 1
2011-2012 Bachelor semestre 2
2011-2012 Bachelor semestre 3
2011-2012 Bachelor semestre 4
2011-2012 Bachelor semestre 5
2011-2012 Bachelor semestre 6
2012-2013 Bachelor semestre 1
2012-2013 Bachelor semestre 2
2012-2013 Bachelor semestre 3
2012-2013 

In [11]:
for year in years:
    for semester in bachelor_semesters:
        print(year, semester, bachelor_datasets[year][semester].shape)

2007-2008 Bachelor semestre 1 (90, 10)
2007-2008 Bachelor semestre 2 (72, 10)
2007-2008 Bachelor semestre 3 (50, 10)
2007-2008 Bachelor semestre 4 (48, 10)
2007-2008 Bachelor semestre 5 (48, 10)
2007-2008 Bachelor semestre 6 (38, 10)
2008-2009 Bachelor semestre 1 (96, 10)
2008-2009 Bachelor semestre 2 (74, 10)
2008-2009 Bachelor semestre 3 (53, 10)
2008-2009 Bachelor semestre 4 (51, 10)
2008-2009 Bachelor semestre 5 (57, 10)
2008-2009 Bachelor semestre 6 (51, 10)
2009-2010 Bachelor semestre 1 (117, 10)
2009-2010 Bachelor semestre 2 (101, 10)
2009-2010 Bachelor semestre 3 (54, 10)
2009-2010 Bachelor semestre 4 (52, 10)
2009-2010 Bachelor semestre 5 (75, 10)
2009-2010 Bachelor semestre 6 (60, 10)
2010-2011 Bachelor semestre 1 (153, 10)
2010-2011 Bachelor semestre 2 (125, 10)
2010-2011 Bachelor semestre 3 (51, 10)
2010-2011 Bachelor semestre 4 (51, 10)
2010-2011 Bachelor semestre 5 (62, 10)
2010-2011 Bachelor semestre 6 (52, 10)
2011-2012 Bachelor semestre 1 (166, 10)
2011-2012 Bachelor s

In [12]:
master_datasets = {}
for year in years:
    master_datasets[year] = {}
for year in years:
    for semester in master_semesters:
        print(year, semester)
        master_datasets[year][semester] = get_dataset(program_values[programs[0]], 
                                                        year_values[year], master_semester_values[semester])

2007-2008 Master semestre 1
2007-2008 Master semestre 2
2007-2008 Master semestre 3
2007-2008 Projet Master automne
2007-2008 Projet Master printemps
2008-2009 Master semestre 1
2008-2009 Master semestre 2
2008-2009 Master semestre 3
2008-2009 Projet Master automne
2008-2009 Projet Master printemps
2009-2010 Master semestre 1
2009-2010 Master semestre 2
2009-2010 Master semestre 3
2009-2010 Projet Master automne
2009-2010 Projet Master printemps
2010-2011 Master semestre 1
2010-2011 Master semestre 2
2010-2011 Master semestre 3
2010-2011 Projet Master automne
2010-2011 Projet Master printemps
2011-2012 Master semestre 1
2011-2012 Master semestre 2
2011-2012 Master semestre 3
2011-2012 Projet Master automne
2011-2012 Projet Master printemps
2012-2013 Master semestre 1
2012-2013 Master semestre 2
2012-2013 Master semestre 3
2012-2013 Projet Master automne
2012-2013 Projet Master printemps
2013-2014 Master semestre 1
2013-2014 Master semestre 2
2013-2014 Master semestre 3
2013-2014 Projet

In [13]:
for year in years:
    for semester in master_semesters:
        print(year, semester, master_datasets[year][semester].shape)

2007-2008 Master semestre 1 (71, 10)
2007-2008 Master semestre 2 (87, 10)
2007-2008 Master semestre 3 (65, 10)
2007-2008 Projet Master automne (1, 10)
2007-2008 Projet Master printemps (6, 10)
2008-2009 Master semestre 1 (60, 10)
2008-2009 Master semestre 2 (64, 10)
2008-2009 Master semestre 3 (48, 10)
2008-2009 Projet Master automne (4, 10)
2008-2009 Projet Master printemps (6, 10)
2009-2010 Master semestre 1 (52, 10)
2009-2010 Master semestre 2 (62, 10)
2009-2010 Master semestre 3 (34, 10)
2009-2010 Projet Master automne (3, 10)
2009-2010 Projet Master printemps (8, 10)
2010-2011 Master semestre 1 (96, 10)
2010-2011 Master semestre 2 (109, 10)
2010-2011 Master semestre 3 (40, 10)
2010-2011 Projet Master automne (3, 10)
2010-2011 Projet Master printemps (7, 10)
2011-2012 Master semestre 1 (102, 10)
2011-2012 Master semestre 2 (123, 10)
2011-2012 Master semestre 3 (67, 10)
2011-2012 Projet Master automne (9, 10)
2011-2012 Projet Master printemps (13, 10)
2012-2013 Master semestre 1 (88

?Q?
Not many students in Master project.
How exactly should be used URL (frames).
"Extra" parametrs when retrieve table.
Can I hardcore params that are the same.